In [ ]:
# import relevant modules
import numpy as np
import gdal
import os
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [ ]:
# Assign working directory (from/to where the images will be read/write)
os.chdir('C:/Users/SNMACIAS/Documents/CIMMYT/COMPASS/B2212/')

In [ ]:
# Set figure size (scattered plot)
#fig_size = 
plt.rcParams['figure.figsize'] = (7, 7)

### A. Import and open image

In [ ]:
# Cut to shapefile
img_in = 'C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/output/q190912dur_transparent_reflectance_all.tif'      # Input file
img_clp = 'test_clip_.tif'                               # Output clipped file
gdal.Warp(img_clp, img_in, dstNodata = -10000, \
          cutlineDSName = 'C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/analysis/test_clip.shp', \
          cropToCutline = True)

In [ ]:
img_clp = 'e190328co1rfl.tif'

In [ ]:
# Use entire mosaic
img_clp = 'C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/output/q190912dur_transparent_reflectance_all.tif'

In [ ]:
# Open the file
fid=gdal.Open(img_clp, gdal.GA_ReadOnly)
rows=fid.RasterYSize
cols=fid.RasterXSize

# Read relevant bands (R and NIR) and store them as arrays
R=fid.GetRasterBand(3).ReadAsArray()
NIR=fid.GetRasterBand(5).ReadAsArray()
del fid

In [ ]:
R.shape, NIR.shape

### B. Level 1 image processing: Deal with homogeneization of bands

In [ ]:
# Assign nan
mask1 = R != -3.3999999521443642e+38                 # Boolean: nodata false, else true
R_a = R*mask1                                        # float and 0's
R_a[R*mask1 == 0] = np.nan                           # float and nan

mask2 =  NIR != -3.3999999521443642e+38              # Boolean: nodata false, else true
NIR_a = NIR*mask2                                    # float and 0's
NIR_a[NIR*mask2 == 0] = np.nan                       # floar and nan

# Eliminate where either in R or NIR there is nan
mask3 = np.where(np.isnan(NIR_a+R_a), True, False)

# Test whether R and NIR have same shape to be able to plot
NIR_a[mask3].shape, R_a[mask3].shape

In [ ]:
# Define soil line
x1, y1 = np.nanquantile(R_a, 0.9995), np.nanquantile(NIR_a, 0.075)       # Bare bright soil (upper right)
x0, y0 = np.nanquantile(R_a, 0.85), np.nanquantile(NIR_a, 0.00005)     # Bare dark soil (lower left)

m = (y1 - y0)/(x1 - x0)               # Set slope of soil line
c = (-x0/(x1-x0))*(y1-y0)+y0          # Set intercept of soil line

# Define full canopy cover
x2, y2 = 0.00862942699342966, 0.658362090587616 # np.nanquantile(R_a, 0.005), np.nanquantile(NIR_a, 0.995) 

x2, y2, x1, y1, x0, y0, m, c

In [ ]:
# Scatter plot
plt.scatter([x0, x1, x2], [y0, y1, y2], s=100, c='black')      # These are the points defining soil line and full canopy pixel
plt.scatter(R_a, NIR_a, c='blue', alpha=0.01)                  # These are the pairs of Red and NIR

plt.legend(loc=0)
plt.xlabel('Red [reflectance]')
plt.ylabel('NIR [reflectance]')
#plt.xlim([0,0.3])
#plt.ylim([0,0.8])

### C. Define PVI for the one pixel and for all pixels

In [ ]:
# PVI fcp (eq. 6 Mass & Rajan 2008)
PVIfcp = (y2 - m * x2 - c) / pow((1+pow(m, 2)), 0.5)

# Calculate PVI for all pixes (eq. 5 Mass & Rajan 2008)
PVIallpix = (NIR_a - m * R_a - c) / pow((1+pow(m, 2)), 0.5)

In [ ]:
# Test only / do not run
mask4 = PVIallpix < 0                               # Boolean: < 0 false, else true
PVIallpix_a = PVIallpix*mask4                       # float and 0's
PVIallpix_a[PVIallpix*mask4 == 0]                   # float and nan

mask5 = PVIallpix_a > 1                             # Boolean: > 1 false, else true
PVIallpix_b = PVIallpix_a*mask5                     # float and 0's
PVIallpix_b[PVIallpix_a*mask5 == 0] = 1.0           # float and nan

### D. Estimate ground cover and create image

In [ ]:
# Prepare image
indataset = gdal.Open(img_clp)
driver = gdal.GetDriverByName("GTiff")                     # Define driver for image creation
dType = gdal.GDT_Float32

projection = indataset.GetProjection()                     # Read projection
geolocation = indataset.GetGeoTransform()                  # Read georeference

In [ ]:
# Calculate gc and write file
outArray = indataset.GetRasterBand(1).ReadAsArray()
outArray = np.divide(PVIallpix, PVIfcp)                    # GC calculation (eq. 7 Mass & Rajan 2008)

img_out = 'e190328co1rfl_gc_nir_q_0995.tif'
outdataset = driver.Create(img_out, cols, rows, 1, dType)  # Image creation
outdataset.SetProjection(projection)                       # Assign projection
outdataset.SetGeoTransform(geolocation)                    # Assign geolocation

outdataset.GetRasterBand(1).WriteArray(outArray)           # Write gc file

In [ ]:
#PVIallpix.shape, outArray.shape, outArray

### Z. Appendix

In [ ]:
# Assign nan
Rnan =  R != -3.4e+38
R[~Rnan] = np.nan

NIRnan =  NIR != -3.4e+38
NIR[~NIRnan] = np.nan

# Eliminate where either in R or NIR there is nan
mask = np.where(np.isnan(R+NIR), np.where(np.isnan(R), NIR, R), R+NIR)

# Test whether R and NIR have same shape to be able to plot
R[np.isnan(mask)].shape, NIR[np.isnan(mask)].shape